Tutorial 1 - Basic Flowsheets
=============================

Introduction
------------

For this tutorial on how to construct simple flowsheets within the IDAES framework, we will use a simple steady-state process of two well-mixed reactors (CSTRs) in series, with the following system reactions occurring in each.

<img src="2cstrs.png">

<center>ethyl acetate + NaOH $\rightarrow$ sodium acetate + ethanol</center>

In this tutorial, you will learn how to:

* import the necessary libraries from Pyomo and IDAES
* create a flowsheet object
* add a property package to a flowsheet
* add unit models to a flowsheet
* connect units together
* set inlet conditions and design variables
* initialize a model
* solve the model and print some results

In this tutorial, we will be working within a Jupyter Notebook, which is an interactive Python environment which allows us to write and execute code as we go. Working in other environments uses the same set of commands as those you will use here, thus the code demonstrated here is applicable no matter what environment you eventually work in.

A completed example of this tutorial as a Python script is available in the same folder as this Notebook.

First Steps
-----------

At this stage, our Jupyter Notebook is a bare Python environment and it does not contain the tools we need to use. To setup the tools we need to import the packages and components we are going to use into our environment.

The first thing we need to do is import some components from Pyomo which will be used in our model:

* `ConcreteModel` will be used to form the basis of our model,
* `SolverFactory` will be used to solve our flowsheet, and
* `Arc` and `TransformationFactory` will be used to connect unit operations together.

Most of these can be imported from `pyomo.environ` using the following format, whilst `Arc` needs to be imported from `pyomo.network`. To do this, enter the following code into the empty cell below.

```py
from pyomo.environ import ConcreteModel, SolverFactory, TransformationFactory
from pyomo.network import Arc
```

In [2]:
from pyomo.environ import ConcreteModel, SolverFactory, TransformationFactory
from pyomo.network import Arc

Next, we need to import the features we are going to use from the IDAES code. The first thing we will need is `FlowsheetBlock`, which is found in `idaes.core`. FlowsheetBlocks are used as the basis for constructing all flowsheets in IDAES, and provides the necessary infrastructure for constructing a flowsheet.

```py
from idaes.core import FlowsheetBlock
```

In [3]:
from idaes.core import FlowsheetBlock

Next, we need to import some models for the thermo-physical and reaction properties of the system we want to model. Within the IDAES modeling framework, thermo-physical and kinetic properties are handled by separate modules referred to as property packages (this is to allow for modularity of property calculations). For this tutorial, two property packages have been prepared for you, so all we need to do is import them (later tutorials will cover how to write your own property packages).

```py
import idaes.property_models.examples.saponification_thermo as thermo_props
import idaes.property_models.examples.saponification_reactions as reaction_props
```

In [4]:
import idaes.property_models.examples.saponification_thermo as thermo_props
import idaes.property_models.examples.saponification_reactions as reaction_props

Finally, we need to import the models for the unit operations we wish to use in our flowsheet. IDAES comes with a library of models for common unit operations, which includes a model for a CSTR.

```py
from idaes.unit_models import CSTR
```

In [5]:
from idaes.unit_models import CSTR

Setting up a model and flowsheet
--------------------------------

Now that we have imported the necessary models and tools, we can begin constructing the model of our process. The first step in constructing a model within the IDAES framework is to create a `ConcreteModel` with which to contain the flowsheet. This is the same as creating a ConcreteModel within Pyomo.

```py
m = ConcreteModel()
```

Next, we add a `FlowsheetBlock` object to our model. `FlowsheetBlock` is an IDAES modeling object which contains all the attributes required of a flowsheet within the IDAES framework (such as the time domain for dynamic models). `FlowsheetBlock` supports a number of different options, which can be used to change the behavior of the model. For the current example we want a steady-state flowsheet, which is done by setting the option `dynamic=False`.

Setting the options for any IDAES component is done using the `default` and `initialize` arguments when creating an instance of the component. For this case, we will use the `default` argument to create a steady-state flowsheet named `fs` as shown below (the `initialize` argument will be covered later):

```py
m.fs = FlowsheetBlock(default={"dynamic": False})
```

This creates a flowsheet block within our model which we can now start to populate with property calculations and unit models. As unit models depend on property packages for part of their construction, we need to add the property packages to our flowsheet first.

In this flowsheet, we have two property packages to add; the thermo-physical property package and the reaction property package. In many cases, reaction properties are dependent on some thermo-physical properties and models for reaction properties are implicitly dependent on the model used to calculate the thermo-physical properties. To maintain this dependency within the IDAES framework, reaction property packages must be linked to a compatible thermo-physical property package.

Thus, we need to add the thermo-physical property package first. This is done by creating an instance of a `PhysicalParameterBlock`, which is part of all IDAES thermo-physical property packages. This block contains all the information required to set up a thermo-physical property package for our flowsheet, and can be passed to reaction packages and unit models in order to set up the property calculations within that model. The actual name of the parameter block will depend upon the property package being used, and users should refer to the package documentation. In this case, the name of the parameter block is `SaponificationParameterBlock`.

To add a property package with the name `thermo_params` to our Flowsheet using the `thermo_props` package we imported earlier, we do the following:

```py
m.fs.thermo_params = thermo_props.SaponificationParameterBlock()
```

Next, we can add the reaction property package and link it to the thermo-physical property package. Linking the thermo-physical properties to the reaction properties is done using an option in the reaction package, similar to the way we set the `dynamic` option in the flowsheet. Similar to thermophysical property packages, users will need to refer to the package documentation for the name of the reaction parameter block, which in this case is `SaponificationReactionParameterBlock`.

```py
m.fs.reaction_params = reaction_props.SaponificationReactionParameterBlock(
                        default={"property_package": m.fs.thermo_params})
```

Now that we have our property package set up, we can add our CSTRs to the flowsheet, which we will call `Tank1` and `Tank2`. Just like `FlowsheetBlocks`, unit models have a number of options available which can be used to change the form of the model and determine what property packages to use. Different unit models will have their own set of options, which will be covered in their documentation.

For this tutorial, we will need to set the following options taken from the CSTR documentation:

* `property_package` specified which thermo-physical property package should be used in the model.
* `reaction_package` specifies which reaction package should be used (only needed in unit operations with reactions).
* `has_equilibrium_reactions` sets whether constraints for enforcing chemical equilibrium should be included. In this example we have no equilibrium reactions, so we will set this to `False`.
* `has_heat_transfer` specifies whether heat of reaction needs to be explicitly accounted for in the energy balance equations. In some cases, heat of reaction will be accounted for implicitly in the specific enthalpy calculations, however this is not the case for this example (you will need to refer to the documentation of each property package to see how heat of reaction is accounted for in each case).
* `has_heat_transfer` specifies whether to include a heat duty term in the energy balance equations. For this tutorial we will include the heat transfer term to demonstrate how to use it.
* `has_pressure_change` specifies whether to include a pressure change term in the momentum balance equations. For this tutorial we will set this to `False`.

To create our CSTRs with the above options, we do the following:

```py
m.fs.Tank1 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_equilibrium_reactions": False,
                           "has_heat_of_reaction": True,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})
m.fs.Tank2 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_equilibrium_reactions": False,
                           "has_heat_of_reaction": True,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})
```

Finally, we need to connect our CSTRs together into a flowsheet. Our CSTRs already contain inlet and outlet objects (Pyomo `Port` objects) which were created when the models were constructed, so all we need to do is connect the outlet of `Tank1` to the inlet of `Tank2`. We do this using Pyomo `Arc` components, which we imported at the beginning of the tutorial.

To create an `Arc` we do the following:

```py
m.fs.stream = Arc(source=m.fs.Tank1.outlet,
                  destination=m.fs.Tank2.inlet)
```

There is one final step in setting up the `Arcs` connecting our flowsheet, which is to expand the `Arcs` into an actual set of `Constraints` in our model. Once all the `Arcs` in the flowsheet have been created, we apply the Pyomo `TransformationFactory` to automatically expand all the `Arcs` for us, as shown below:

```py
TransformationFactory("network.expand_arcs").apply_to(m)
```

In this case, we apply the `network.expand_arcs` transformation to our entire model (`m`).

Setting Operating Conditions
----------------------------

Now that our flowsheet is fully constructed, we can start to specify the conditions we wish to simulate. The first conditions to set are the conditions of the feed stream to `Tank1`. The state variables we need to define are determined by the thermo-physical property package we are using, and in this case they are the following:


* volumetric flowrate = 1.0 [m^3/s]
* concentration of H2O = 55388.0 [mol/m^3]
* concentration of NaOH = 100.0 [mol/m^3]
* concentration of ethylacetate = 100.0 [mol/m^3]
* concentration of ethanol = 0.0 [mol/m^3]
* concentration of sodium acetate = 0.0 [mol/m^3]
* temperature = 303.15 [K]
* pressure = 101325.0 [Pa]

The IDAES modeling framework uses a set of standard names for different quantities, which can be found in the IDAES documentation. The standard names for each quantity we need to fix are given below, along with the format for how to fix value of each quantity.

```py
m.fs.Tank1.inlet.flow_vol[0].fix(1.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "H2O"].fix(55388.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "NaOH"].fix(100.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "EthylAcetate"].fix(100.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "SodiumAcetate"].fix(0.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "Ethanol"].fix(0.0)
m.fs.Tank1.inlet.temperature[0].fix(303.15)
m.fs.Tank1.inlet.pressure[0].fix(101325.0)
```

Additionally, we need to specify some design conditions for the system – in this case the volume and heat duty of both tanks. Let us fix the volume of each tank to be 1 $m^3$ and the heat duty to be 0 $J/s$. The variable names are "volume" and "heat_duty".

```py
m.fs.Tank1.volume.fix(1.0)
m.fs.Tank1.heat_duty.fix(0.0)

m.fs.Tank2.volume.fix(1.0)
m.fs.Tank2.heat_duty.fix(0.0)
```

In this case you will note that we did not include the time index on the volume and heat duty variables, even though both are indexed by time. In cases like this where we have an indexed variable which we wish to fix at all indices, we can drop the index and the variable will be fixed to the given value at all indices.

Initializing and Solving the Model
----------------------------------

Now that the model has been constructed and the inlet and design conditions have been specified, we can now work on solving the model. However, most process engineering models cannot be solved in a single step, and require some degree of initialization to get to a solvable state. The models within the IDAES model library contain pre-built initialization routines which can be used to get each model to a solvable state. For this tutorial, we will use a manual sequential modular type approach to initializing our flowsheet using these pre-built methods.

We will begin with initializing `Tank1`, as it is the first unit in our flowsheet. The initialization routine for a CSTR expects the conditions of the inlet stream to be provided as initial guesses along with any design conditions required to have zero degrees of freedom. However, as we have already provided (fixed) all of these, there is no need to provide additional guesses for these and the initialization routine will make use of the specified value automatically.

The initialization routines also require a non-linear solver to be available to solve the model. This tutorial assumes that you have IPOPT installed, however you can substitute this for other NLP solvers you may have available. In order to do this, you can set the solver keyword when calling the initialization routine with the name of your NLP solver (e.g. `solver=’ipopt’`).

To initialize `Tank1`, we do the following:

```py
m.fs.Tank1.initialize()
```

Now that `Tank1` has been initialized, we can move onto `Tank2`. As the inlet to `Tank2` is not fixed, we need some initial guesses for the inlet conditions. For now, this is a manual process where the values are provided to the initialization routine through the `state_args` argument (which is a Python `dict`). For now, lets use the feed conditions as out initial guesses (in later tutorials we will see how to use the outlet of one unit to initialize the next).

```py
m.fs.Tank2.initialize(state_args={
        "flow_vol": 1.0,
        "conc_mol_comp": {"H2O": 55388.0,
                          "NaOH": 100.0,
                          "EthylAcetate": 100.0,
                          "SodiumAcetate": 0.0,
                          "Ethanol": 0.0},
        "temperature": 303.15,
        "pressure": 101325.0})
```

Solving the Model
--------------------------

At this point, our model should now be initialized and ready to solve. In order to do this, we need to create a `solver` using Pyomo's `SolverFactory`:

```py
solver = SolverFactory('ipopt')
```

Just like in Pyomo, solver options can be provided as well by attaching a dictionary of keywords to the solver object, however these are not needed for this tutorial. Once the solver object is created, we can call it to solve the model and return the results object. The `tee` argument will output the solver messages to the screen so you can see the progress of the solver.

```py
results = solver.solve(m, tee=True)
```

Hopefully you get a message from the solver saying "Optimal Solution Found" (or something similar). This indicates that the solver managed to find a feasible solution to the problem and you have some results to work with. If you see some other message, then you will need to go back through your model and make sure that everything is correct.

Viewing the Results
-----------------------------

Now that we have some results, lets have a look at the outputs. First, let's print the `results` object that was returned by the solver.

```py
print(results)
```

This output shows some basic details of the problem that was solved, including the number of variables and constraints (it is always good to check that the degrees of freedom match). In this case you should see 40 variables and constraints. You might also see 1 objective (e.g. if using IPOPT) - whilst we didn't add an objective to our model, the solver sometimes adds a trivial objective to satisfy its internal algorithms.

Next, let us look at the solution of the outlet of the two tanks. To do this, we can `display` the outlets of each tank.

```py
print("Tank 1 Outlet")
m.fs.Tank1.outlet.display()
print()
print("Tank 2 Outlet")
m.fs.Tank2.outlet.display()
```

If all has gone well, you should see the following results.

**Tank 1**
* flow_vol = 1.0 [m^3/s]
* conc_mol_comp["SodiumAcetate"] = 75.704 [mol/m^3]
* conc_mol_comp["Ethanol"] = 75.704 [mol/m^3]
* conc_mol_comp["NaOH"] = 24.296 [mol/m^3]
* conc_mol_comp["EthylAcetate"] = 24.296 [mol/m^3]
* conc_mol_comp["H2O"] = 55388.0 [mol/m^3]
* pressure = 101325 [Pa]
* temperature = 304.04 [K]

**Tank 2**
* flow_vol = 1.0 [m^3/s]
* conc_mol_comp["SodiumAcetate"] = 89.628 [mol/m^3]
* conc_mol_comp["Ethanol"] = 89.628 [mol/m^3]
* conc_mol_comp["NaOH"] = 10.372 [mol/m^3]
* conc_mol_comp["EthylAcetate"] = 10.372 [mol/m^3]
* conc_mol_comp["H2O"] = 55388.0 [mol/m^3]
* pressure = 101325 [Pa]
* temperature = 304.20 [K]